---

## EXERCISE1: Burglary network

Implement the *Prior Sampling* algorithm to do approximate inference on last week's Burglary network.

<img src='https://drive.google.com/uc?id=11xqbchd4TCFSzdLVxNgr4SoeGfrSoqGO'>

Verify that the algorithm can correctly approximate the probability $P(j, m, a, \neg b, \neg e) = 0.00063$

Try different numbers of samples (e.g. $N = 10, 100, 1000, 10000$) and compare the results.

---

Funzione per generare campioni data una distribuzione e lo stato dei parenti

In [ ]:
import numpy as np
import random as rnd

tr, fa = 0, 1

#Pdist: the probability distribution from the CPT
# Parentes[]: if available the values of the Parents event (i.e., t or f)

def samplegen(Pdist, Parents = []):       # Parents = [] represents the state of the parents
	assert len(Parents) < len(Pdist.shape)
	if rnd.random() < Pdist[tr][tuple(Parents)]:
		return tr
	return fa

#	It works as follows:
#If Parents is empty (no condition), it generates a value based on the marginal
#probabilities in Pdist.
#If Parents is provided, it selects the conditional probability corresponding
#to the parent values and determines the outcome.

Probabilities

In [ ]:
P_B = np.array([0.001, 0.999])
P_E = np.array([0.002, 0.998])
P_A_BE = np.array([[[0.95, 0.94], [0.29, 0.001]], [[0.05, 0.06], [0.71, 0.999]]])
P_J_A = np.array([[0.90, 0.05], [0.10, 0.95]])
P_M_A = np.array([[0.70, 0.01], [0.30, 0.99]])

Computing the probability $P(j, m, a, \neg b, \neg e) = 0.00063$

$P(\neg e) P(\neg b) P(a| \neg b, \neg e) P(m|a) P(j|a)$


Function to compute the estimations of probabilities from the probability distribution, if true or false is wanted and the state of the parents(optional)

In [ ]:
def samp(p, vf, g = []):
  n = 100000
  falso = 0
  vero = 0
  for i in range(n):
    s = samplegen(p, g)
    if (s == 0):
      vero += 1
    elif (s == 1):
      falso += 1
  if (vf == 't'):
    return vero/n
  elif(vf == 'f'):
    return falso/n

$P(\neg b)$

In [ ]:
p1 = samp(P_B, 'f')
print('probabilità stimata = ', p1,
      '\nvera probabilità = ', P_B[fa])

probabilità stimata =  0.99912 
vera probabilità =  0.999


$P(\neg e)$

In [ ]:
p2 = samp(P_B, 'f')
print('probabilità stimata = ', p2,
      '\nvera probabilità = ', P_E[fa])

probabilità stimata =  0.99885 
vera probabilità =  0.998


$P(a| \neg b, \neg e)$

In [ ]:
p3 = samp(P_A_BE, 't', [fa, fa])
print('probabilità stimata = ', p3,
      '\nvera probabilità = ', P_A_BE[tr, fa, fa])

probabilità stimata =  0.00097 
vera probabilità =  0.001


$P(m|a)$

In [ ]:
p4 = samp(P_M_A, 't', [tr])
print('probabilità stimata = ', p4,
      '\nvera probabilità = ', P_M_A[tr, tr])

probabilità stimata =  0.69985 
vera probabilità =  0.7


$P(j|a)$

In [ ]:
p5 = samp(P_J_A, 't', [tr])
print('probabilità stimata = ', p5,
      '\nvera probabilità = ', P_J_A[tr, tr])

probabilità stimata =  0.90043 
vera probabilità =  0.9


Quindi facendo il prodotto per ottenere la probabilità complessiva otteniamo

In [ ]:
p = p1 * p2 * p3 *p4 * p5
print('probabilità stimata = ', p,
      '\nvera probabilità =', P_E[fa] * P_B[fa] * P_A_BE[tr, fa, fa] *
                              P_M_A[tr, tr] * P_J_A[tr, tr])

probabilità stimata =  0.0006100207162874959 
vera probabilità = 0.0006281112599999998


Increasing $n$, the accuracy improves

---

## EXERCISE2: Pomegranate for Day 2

Use <tt>pomegranate</tt> to calculate the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2.

What is the filtered probability of rain on Day 2 when we see <tt>not umbrella</tt> on Day 1? (But we still see it in Day 2)

How about if we just have no information about Umbrellas on Day 1 (But we still see the umbrella in Day 2)?

In [2]:
!pip install numpy
!pip install pomegranate==0.15.0

  Using cached pomegranate-0.15.0.tar.gz (5.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pomegranate: filename=pomegranate-0.15.0-cp310-cp310-linux_x86_64.whl size=21355230 sha256=1c5aa776661086a2b7684b5057cc15127af55bfa43fb84a92f50e5dc3def4f59
  Stored in directory: /root/.cache/pip/wheels/55/44/b9/c0399e42f50907ada5da7d160bcd1465a39f36b935d9e59f5d
Successfully built pomegranate


Use <tt>pomegranate</tt> to calculate the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2.


In [3]:
from pomegranate import *

# Variables are RainN and UmbrellaN+1 for N = 0, 1, ...
# We have a prior for Rain0, two values 'y'es and 'n'o:
Rain0   = DiscreteDistribution({'y': 0.5, 'n': 0.5})

# Transition model
#
# Conditional distribution relating RainN and RainN+1. Notation for
# the conditional probability table is:
#
# [ 'RainN', 'RainN+1', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Note that we have to
# repeat the transition model for each pair of states
Rain1 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain0])

Rain2 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain1])

# Sensor model
#
# Conditional distribution relating Rain and Umbrella:
#
# [ 'Umbrella', 'Rain', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Values for Umbrella are 'y'es and 'n'o.
# Again we have to enter the table for each day.
Umbrella1 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain1])

Umbrella2 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain2])
#
# The whole network has five nodes:
s1 = Node(Rain0, name="Rain0")
s2 = Node(Rain1, name="Rain1")
s3 = Node(Umbrella1, name="Umbrella1")
s4 = Node(Rain2, name="Rain2")
s5 = Node(Umbrella2, name="Umbrella2")
# Create a network that includes nodes and edges between them:
model = BayesianNetwork("Umbrella Network")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s2)
model.add_edge(s2, s3)
model.add_edge(s2, s4)
model.add_edge(s4, s5)
# Fix the model structure
model.bake()

In [4]:
# Conditional probability table
Rain3 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain2])
# Node
s6 = Node(Rain3, name="Rain3")
# State
model.add_states(s6)
# Edge
model.add_edge(s4, s6)
# Fix the model structure
model.bake()

In [8]:
#QUESTION 2.1
# Set Umbrella1 and Umbrella 2 to 'y'
scenario = [[None, None, 'y', None, 'y', None]]
# Run the model
results =  model.predict_proba(scenario)
# And the probability of rain on Day 2:
print(results[0][3].items())

# Rain0 → scenario[0][0]
# Rain1 → scenario[0][1]
# Umbrella1 → scenario[0][2]
# Rain2 → scenario[0][3]
# Umbrella2 → scenario[0][4]
# Rain3 → scenario[0][5]


(('y', 0.8833570412517777), ('n', 0.11664295874822232))


In [6]:
#QUESTION 2.2
# Don't see Umbrella1 on Day 1 (i.e., set to 'n'), set Umbrella 2 to 'y'
scenario = [[None, None, 'n', None, 'y', None]]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 2:
print(results[0][3].items())

(('y', 0.7027707808564231), ('n', 0.2972292191435768))


In [7]:
# QUESTION 2.3
# No infos about umbrellas only Rain on day2 equal
scenario = [[None, None, None, None, 'y', None]]
# Run the model
results =  model.predict_proba(scenario)
#print(results)
# Ask for the probability of rain on Day 2:
print(results[0][3].items())

(('y', 0.8181818181818179), ('n', 0.1818181818181821))
